### Aprendizado Ensemble - Florestas Aleatórias

1. Desenvolvimento do algoritmo de indução de uma árvore de decisão, usando como critério de seleção de atributos para divisão de nós o **Ganho de Informação (baseado no conceito de entropia)**. **Tratando tanto atributos categóricos quanto numéricos.**

In [9]:
from collections import Counter, defaultdict
from functools import partial
import math, random

def entropy(class_probabilities):
    """given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p, 2) for p in class_probabilities if p)

def gini_criterion(data, labels):
  """ Gini Index
  Parameters
  ----------
  data: numpy array-like = [n_samples, n_features]
  labels: numpy array-like, shape = [n_samples]
  
  Return
  ------
  gini: float
  """
  classes = np.unique(labels)
  
  s = 0
  for c in classes:
    p = np.mean(labels == c)
    s += p * (1 - p)
    
  return s

def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    print('[INFO] entropy', entropy(probabilities))
    return entropy(probabilities)

def partition_entropy(subsets):
    """find the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets)

    #print('[INFO] subset: ', subsets)
    return sum( data_entropy(subset) * len(subset) / total_count
                for subset in subsets )

def group_by(items, key_fn):
    """returns a defaultdict(list), where each input item
    is in the list whose key is key_fn(item)"""
    groups = defaultdict(list)
    for item in items:
        key = key_fn(item)
        groups[key].append(item)
    return groups

def partition_by(inputs, attribute):
    """returns a dict of inputs partitioned by the attribute
    each input is a pair (attribute_dict, label)"""
    return group_by(inputs, lambda x: x[0][attribute])

def partition_entropy_by(inputs, attribute):
    """computes the entropy corresponding to the given partition"""
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values())

def build_tree_id3(inputs, split_candidates=None):

    # if this is our first pass,
    # all keys of the first input are split candidates
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()
    #print(split_candidates)

    # count Trues and Falses in the inputs
    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues

    if num_trues == 0:                  # if only Falses are left
        return False                    # return a "False" leaf

    if num_falses == 0:                 # if only Trues are left
        return True                     # return a "True" leaf

    if not split_candidates:            # if no split candidates left
        return num_trues >= num_falses  # return the majority leaf

    # otherwise, split on the best attribute
    best_attribute = min(split_candidates,
        key=partial(partition_entropy_by, inputs))
    print('[INFO] best_attribute: ', best_attribute)

    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates
                      if a != best_attribute]

    # recursively build the subtrees
    subtrees = { attribute : build_tree_id3(subset, new_candidates)
                 for attribute, subset in partitions.items() }

    #subtrees[None] = num_trues > num_falses # default case

    return (best_attribute, subtrees)

if __name__ == "__main__":

    # inputs contendo tupulas contendo dicionarios
    inputs = [
        ({'Tempo':'Ensolarado','Temperatura':'Quente','Umidade':'Alta','Ventoso':'no'},   False),
        ({'Tempo':'Ensolarado','Temperatura':'Quente','Umidade':'Alta','Ventoso':'yes'},  False),
        ({'Tempo':'Nublado','Temperatura':'Quente','Umidade':'Alta','Ventoso':'no'},     True),
        ({'Tempo':'Chuvoso','Temperatura':'Amena','Umidade':'Alta','Ventoso':'no'},  True),
        ({'Tempo':'Chuvoso','Temperatura':'Fria','Umidade':'Normal','Ventoso':'no'},      True),
        ({'Tempo':'Chuvoso','Temperatura':'Fria','Umidade':'Normal','Ventoso':'yes'},    False),
        ({'Tempo':'Nublado','Temperatura':'Fria','Umidade':'Normal','Ventoso':'yes'},        True),
        ({'Tempo':'Ensolarado','Temperatura':'Amena','Umidade':'Alta','Ventoso':'no'}, False),
        ({'Tempo':'Ensolarado','Temperatura':'Fria','Umidade':'Normal','Ventoso':'no'},      True),
        ({'Tempo':'Chuvoso','Temperatura':'Amena','Umidade':'Normal','Ventoso':'no'}, True),
        ({'Tempo':'Ensolarado','Temperatura':'Amena','Umidade':'Normal','Ventoso':'yes'},True),
        ({'Tempo':'Nublado','Temperatura':'Amena','Umidade':'Alta','Ventoso':'yes'},    True),
    ]
    
    print("[INFO] Construindo a árvore:")
    tree = build_tree_id3(inputs)
    print(tree)

[INFO] Construindo a árvore:
[INFO] entropy 0.9709505944546686
[INFO] entropy 0.0
[INFO] entropy 0.8112781244591328
[INFO] entropy 0.9182958340544896
[INFO] entropy 0.7219280948873623
[INFO] entropy 0.8112781244591328
[INFO] entropy 1.0
[INFO] entropy 0.6500224216483541
[INFO] entropy 0.863120568566631
[INFO] entropy 0.9709505944546686
[INFO] best_attribute:  Tempo
[INFO] entropy 0.0
[INFO] entropy 1.0
[INFO] entropy 0.0
[INFO] entropy 0.0
[INFO] entropy 0.0
[INFO] entropy 0.9182958340544896
[INFO] entropy 1.0
[INFO] best_attribute:  Umidade
[INFO] entropy 0.0
[INFO] entropy 1.0
[INFO] entropy 0.0
[INFO] entropy 0.9182958340544896
[INFO] entropy 0.0
[INFO] entropy 0.0
[INFO] best_attribute:  Ventoso
('Tempo', {'Ensolarado': ('Umidade', {'Alta': False, 'Normal': True}), 'Nublado': True, 'Chuvoso': ('Ventoso', {'no': True, 'yes': False})})


2. Uma função para percorrer a árvore de decisão treinada e realizar a classificação de uma nova instância (do conjunto de teste);

In [2]:
def classify(tree, input):
    """classify the input using the given decision tree"""

    # if this is a leaf node, return its value
    if tree in [True, False]:
        return tree

    # otherwise find the correct subtree
    attribute, subtree_dict = tree

    subtree_key = input.get(attribute)  # None if input is missing attribute

    if subtree_key not in subtree_dict: # if no subtree for key,
        subtree_key = None              # we'll use the None subtree

    subtree = subtree_dict[subtree_key] # choose the appropriate subtree
    return classify(subtree, input)     # and use it to classify the input

print('\n')
print('[INFO] Dados de amostras (ultimas duas linhas) para teste de resultado:')
print("Nublado / Quente / Normal / Falso ->", classify(tree,
    { "Tempo" : "Nublado",
      "Temperatura" : "Quente",
      "Umidade" : "Normal",
      "Ventoso" : "no"} ))

print("Chuvoso / Amena / Alta / Verdadeiro ->", classify(tree,
    { "Tempo" : "Chuvoso",
      "Temperatura" : "Amena",
      "Umidade" : "Alta",
      "Ventoso" : "yes"} ))



[INFO] Dados de amostras (ultimas duas linhas) para teste de resultado:
Nublado / Quente / Normal / Falso -> True
Chuvoso / Amena / Alta / Verdadeiro -> False


3. O mecanismo de bootstrap (amostragem com reposição) para geração de subconjuntos a partir do conjunto de dados de treinamento originais. Cada bootstrap será utilizado para o treinamento de uma árvore no aprendizado ensemble

4. O mecanismo de amostragem de m atributos a cada divisão de nó, a partir dos quais será selecionado o melhor atributo de acordo com o critério de Ganho de Informação

5. O treinamento de um ensemble de árvores de decisão, adotando os mecanismos de bootstrap e seleção de atributos com amostragem, como mencionados acima

6. O mecanismo de votação majoritária entre as múltiplas árvores de decisão no ensemble, para classificação de novas instâncias utilizando o modelo de Florestas Aleatórias

7. A técnica de validação cruzada (cross-validation) estratificada, para avaliar poder de generalização do modelo e a variação de desempenho de acordo com diferentes valores para os parâmetros do algoritmo (ex., número de árvores no ensemble)

8. Avaliação do impacto do número de árvores no desempenho do ensemble